In [3]:
#class & function definitions 
import re as regex_re
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import subprocess
from threading import Thread


start = time.time()
path = os.getcwd()

class LEVEL_utilizer:
    def __init__(self):
        self.energy = []
        self.FCF_values = []
        self.x=np.empty(1);
        
        
    
    def readOutput(self, corenumber):
        filename = f"{corenumber}\pro.lvlout"   
        with open(filename, 'r') as file:
            for line in file.readlines():
                if line.startswith(" FCF"):
                    x = regex_re.split("\s+", line)
                    FCF = x[2].replace("D", "e")
                    FCF = float(FCF)
                    dE = x[6]
                    dE = float(dE) * -1
                    self.energy.append(dE)
                    self.FCF_values.append(FCF)
            
            
    def prepareXAxis(self,step):
        energy_min = min(self.energy) - 50
        energy_max = max(self.energy) + 50
        self.x = np.arange(energy_min, energy_max, step)
        
    def gaussianConv(self,tempXAxis, max_pos, FCF, sigmaConv):
        result = np.zeros(tempXAxis.shape)        
        for i,energ in enumerate(tempXAxis):        
            contrib = FCF * np.exp((-(energ-max_pos)**2)/(2 * sigmaConv**2))        
            result[i] = contrib        
        return result
    
    def convolveAll(self,sigmaConv):
        resultList = np.zeros(self.x.shape)
        for i,FCF in enumerate(self.FCF_values):
            contrib = self.gaussianConv(self.x, self.energy[i], self.FCF_values[i], sigmaConv) 
            resultList = resultList + contrib
        return resultList;
    
    @staticmethod
    def calculate_derivative(x, y):
        derivative_y = []
        for i in range(len(y) - 1):
            value = (y[i + 1] - y[i - 1]) / (x[i + 1] - x[i - 1])
            derivative_y.append(value)
        derivative_y.append(derivative_y[len(derivative_y)-1])
        return derivative_y

    @staticmethod
    def get_peaks(x, y, derivative):
        peaks = []
        for i in range(1, len(derivative)):
            if derivative[i] * derivative[i - 1] < 0:
                if np.sign(derivative[i - 1]) == 1 and np.sign(derivative[i]) == -1:
                    peaks.append((x[i], y[i]))

        return peaks

     #refParams=  SolutionParameters(379.5, 3.605, 1.234, 0.08)
    
class RandomParameters:
    def __init__(self):
        self.de_range = (370, 390)
        self.re_range = (3.5, 3.7)
        self.beta_range = (1.10, 1.4)
        self.beta2_range = (0, 0.1)
        self.beta3_range = (0, 0)
        self.de = np.random.uniform(self.de_range[0], self.de_range[1])
        self.re = np.random.uniform(self.re_range[0], self.re_range[1])
        self.beta = np.random.uniform(self.beta_range[0], self.beta_range[1])
        self.beta2 = np.random.uniform(self.beta2_range[0], self.beta2_range[1])
        self.beta3 = np.random.uniform(self.beta3_range[0], self.beta3_range[1])

    def __str__(self):
        return f"de: {self.de:.3f}, re: {self.re:.3f}, beta: {self.beta:.3f}, beta2: {self.beta2:.4f}"


class SolutionParameters:
    def __init__(self, de, re, beta, beta2=0, beta3=0):
        self.de = de
        self.re = re
        self.beta = beta
        self.beta2 = beta2
        self.beta3 = beta3

    def __str__(self):
        return f"de: {self.de:.3f}, re: {self.re:.3f}, beta: {self.beta:.3f}, beta2: {self.beta2:.4f}"


class Spectrum:
    def __init__(self, spectrum_level, spectrum_bcont, score_level, score_bcont, levMultFactor, params: SolutionParameters = None):
        
        if params is None:            
            random_params = RandomParameters()
            self.params = SolutionParameters(random_params.de, random_params.re, random_params.beta, )
        if spectrum_level is None:
            spectrum_level = []
        if spectrum_bcont is None:
            spectrum_bcont = []
        self.spectrum_level = spectrum_level
        self.spectrum_bcont = spectrum_bcont
        self.score_level = score_level
        self.score_bcont = score_bcont
        self.score = self.score_level * levMultFactor + self.score_bcont;
        self.params = params
        

    def __str__(self):
        return f"Sc. lev.: {self.score_level:.3f}, Sc. bcnt: {self.score_bcont:.3f}, Sc: {self.score:.3f}, Par: {self.params}"
    
class Solution:
    def __init__(self, reference_spectrum_level, reference_spectrum_bcont,referenceParams):
        self.random_params = RandomParameters()
        self.reference_spectrum_level = reference_spectrum_level
        self.reference_spectrum_bcont = reference_spectrum_bcont
        self.levelMultFactor=len(ref_spectrum_bcont)/len(ref_spectrum_level)
        #self.levelMultFactor=1.0;
        
        self.reference_params_bcont = referenceParams;
        self.mu=0;
        self.sigma = 10  #to jest sigma do rozkładu Gaussa!!!
        self.sigmaLevel=10  # a ta do levela
        
        self.number_of_best_spectrum = 10
        self.number_of_maxima = 30
        self.population_size = 20
        self.generations = 10
        self.bestResults = "xxxxxxyyyyy";
        self.temp_spectrum = self.get_maxima(self.random_params.de, self.random_params.re, self.random_params.beta, self.random_params.beta2, self.random_params.beta3,self.number_of_maxima,1, self.sigmaLevel)
        self.score = self.calculate_score(self.reference_spectrum_level, self.temp_spectrum)
        
    def __str__(self):
        print(f'Reference level: {self.reference_spectrum_level}')
        print(f'Reference bcont: {self.reference_spectrum_bcont}')
        print(f'temp: {self.temp_spectrum}')
        return f'Score: {self.score}'
        
    @staticmethod
    def get_maxima(de, re, beta, beta2, beta3, number_maxima, corenumber, sigmaLevel):
        threshold = 0        
         # Read data from file
        with open(f"{corenumber}\pro.lvlin", "r") as f:
            lines = f.readlines()

        params_1 = regex_re.split("\s+", lines[38])
        params_2 = regex_re.split("\s+", lines[39])
        params_1[0] = str(de)
        params_1[1] = str(re)
        params_1[2] = str(re)
        params_2[0] = f'\n{str(beta)}   {str(beta2)}   {str(beta3)}'
        lines[38] = "   ".join(params_1)
        lines[39] = f'\n{str(beta)}     {(beta2)} \n'
        #print("beta 2 = {}".format(beta2))

        # Write data to file
        with open(f"{corenumber}\pro.lvlin", "w") as f:
            f.writelines(lines)
        # Run bc.bat
        subprocess.run(f"{path}\\{corenumber}\lp.bat", stdout=subprocess.DEVNULL, cwd=f'{path}\\{corenumber}')
        subprocess.CompletedProcess(args=f"{path}\\{corenumber}\lp.bat", returncode=0)

        LU = LEVEL_utilizer();    
        LU.readOutput(corenumber)
        LU.prepareXAxis(0.05)
        
        spectr1= LU.convolveAll(sigmaLevel)
        derivative = LEVEL_utilizer.calculate_derivative(LU.x, spectr1)
        peaks = LEVEL_utilizer.get_peaks(LU.x, spectr1, derivative)
        filtered_peaks = Solution.filter_peaks(peaks, threshold)
        filtered_peaks_difference = []
        for i, peak in enumerate(filtered_peaks):
            if i!=0:               
                filtered_peaks_difference.append(peak - filtered_peaks[1])
            if number_maxima != 0:
                if i >= number_maxima:
                    break
        filtered_peaks_difference=np.round(filtered_peaks_difference,4)
        return filtered_peaks_difference
    
    @staticmethod
    def get_maxima_bcont(de, re, beta, beta2, core_number, number_maxima=0):
        threshold = 200

        # Read data from file
        with open(f"{core_number}\\bc.in", "r") as f:
            lines = f.readlines()

        # set new values
        lines[46] = '3 0 4 0 1 		 % Final State: FSTYPE=read-in, OMEGA, NFSPRM, VLIMF, XCOORD\n'
        lines[47] = str(de) + "\n"
        lines[48] = str(re) + "\n"
        lines[49] = str(beta) + "\n"
        lines[50] = str(beta2) + "\n"

        # Write data to file
        with open(f"{core_number}\\bc.in", "w") as f:
            f.writelines(lines)

        # Run bc.bat
        subprocess.run(f"{path}\\{core_number}\\bc.bat", stdout=subprocess.DEVNULL, cwd=f'{path}\\{core_number}')
        subprocess.CompletedProcess(args=f"{path}\\{core_number}\\bc.bat", returncode=0)

        data = []
        with open(f"{path}\\{core_number}\\bc.out", "r") as file:
            lines = file.readlines()
            iter = 0;
            for line in lines:
                if "Calculated Transition Intensity Coefficients" in line:
                    iter = iter + 3;
                    break
                else:
                    iter = iter + 1;
            for line in lines[iter:]:
                if "=====" in line:
                    break
                columns = line.split()
                x = float(columns[0])
                y = float(columns[2])
                data.append((x, y))

        # filter out zeros
        data = [(x, y) for x, y in data if x != 0]

        # find first index with y > 1
        start_idx = next((i for i, (_, y) in enumerate(data) if y > 1), 0)

        # shift start_idx to the left by 200
        start_idx = max(start_idx - 200, 0)

        x_values = [item[0] for item in data[start_idx:]]
        y_values = [item[1] for item in data[start_idx:]]
        derivative = Solution.calculate_derivative(x_values, y_values)
        peaks = Solution.get_peaks(x_values, y_values, derivative)
        filtered_peaks = Solution.filter_peaks(peaks, threshold)

        # get distance between each peak and first peak
        filtered_peaks_difference = []
        for i, peak in enumerate(filtered_peaks):
            filtered_peaks_difference.append(peak - filtered_peaks[0])
            if number_maxima != 0:
                if i >= number_maxima:
                    break

        return filtered_peaks_difference
        #filtered_peaks_difference
    
    @staticmethod
    def get_peaks(x, y, derivative):
        peaks = []
        for i in range(1, len(derivative)):
            if derivative[i] * derivative[i - 1] < 0:
                if np.sign(derivative[i - 1]) == 1 and np.sign(derivative[i]) == -1:
                    peaks.append((x[i], y[i]))
        return peaks
    
    @staticmethod
    def filter_peaks(peaks, threshold):
        return [peak[0] for peak in peaks if peak[1] > threshold]
    
    @staticmethod
    def calculate_score(spectrum_ref, spectrum_target):
        if len(spectrum_ref) != len(spectrum_target):
            #print("Problem: ",len(spectrum_ref),len(spectrum_target))
            return float('inf')
        total = 0
        for i in range(len(spectrum_ref)):
            temp = np.abs(spectrum_ref[i] - spectrum_target[i])
            total += temp
        return total
    
    @staticmethod
    def calculate_derivative(x, y):
        derivative_y = []
        for i in range(len(y) - 1):
            value = (y[i + 1] - y[i - 1]) / (x[i + 1] - x[i - 1])
            derivative_y.append(value)
        return derivative_y
    
    def generate_new_spectrum(self, params: SolutionParameters, number_of_maxima,coreNr):           
        spectrum_level = self.get_maxima(params.de, params.re, params.beta, params.beta2,params.beta3, number_of_maxima, coreNr, self.sigmaLevel)
        score_level = self.calculate_score(self.reference_spectrum_level, spectrum_level)       
        spectrum_bcont = self.get_maxima_bcont(params.de, params.re, params.beta, params.beta2, coreNr, number_of_maxima)
        score_bcont = self.calculate_score(self.reference_spectrum_bcont, spectrum_bcont)       
        return Spectrum(spectrum_level, spectrum_bcont, score_level, score_bcont, self.levelMultFactor,  params)

    def crossover(self, parent1: Spectrum, parent2: Spectrum) -> SolutionParameters:
        deratio=np.random.rand();
        de = deratio*parent1.params.de + (1-deratio)*parent2.params.de
        reratio=np.random.rand();
        re = reratio*parent1.params.re + (1-reratio)*parent2.params.re
        betaratio=np.random.rand();
        beta = betaratio*parent1.params.beta + (1-betaratio)*parent2.params.beta
        beta2ratio=np.random.rand();
        beta2 = beta2ratio*parent1.params.beta2 + (1-beta2ratio)*parent2.params.beta2
        return SolutionParameters(de, re, beta, beta2)

    def mutation(self, params: SolutionParameters) -> SolutionParameters:
        de = params.de *(1+ np.random.normal(-0.01, 0.01))
        re = params.re *(1+ np.random.normal(-0.01, 0.01))
        beta = params.beta *(1+ np.random.normal(-0.01, 0.01))
        beta2 = params.beta2 *(1+ np.random.normal(-0.01, 0.01))
        beta2=np.abs(beta2)
        return SolutionParameters(de, re, beta, beta2)

    def get_best_spectrums(self, population, number_of_spectrums):
        population = [spectrum for spectrum in population if spectrum.score != float('inf')]
        population.sort(key=lambda x: x.score)
        return population[:number_of_spectrums]

    def selection(self, population):
        new_population = []
        best = self.get_best_spectrums(population, self.number_of_best_spectrum)
        new_population.extend(best)
        while len(new_population) < self.population_size:
            new_population.append(np.random.choice(population))
        return new_population
    
    def generateInitialPop(self,partPopSize,populationCores,coreNumber):
        #print("generate initial pop core ",coreNumber)
        population = []
        for i in range(partPopSize):
            params = RandomParameters()
            #print(params)
            spectrum = self.generate_new_spectrum(SolutionParameters(params.de, params.re, params.beta, params.beta2),
                                                  self.number_of_maxima, coreNumber)
            population.append(spectrum)
        populationCores[coreNumber]=population;

    def innerEvolution(self, partPopSize, oldPopulation, newPopulationCores, coreNumber):
            tempNewPopulation=[];
            #print("inner evolution ", coreNumber, self.mu, self.sigma)
            for j in range(partPopSize):                
                while True:
                    s1 = int(np.floor(np.abs(np.random.normal(self.mu, self.sigma))))  
                    s2 = int(np.floor(np.abs(np.random.normal(self.mu, self.sigma))))
                    if s1<len(oldPopulation)-1 and s2<len(oldPopulation)-1:
                        break                
                parent1 = oldPopulation[s1]
                parent2 = oldPopulation[s2]
                child_params = self.crossover(parent1, parent2)
                child_params = self.mutation(child_params)  
                #print(child_params)
                spectrum = self.generate_new_spectrum(child_params, self.number_of_maxima,coreNumber)
                tempNewPopulation.append(spectrum)
            newPopulationCores[coreNumber]=tempNewPopulation;
            
    def generate_spectrum_line(self, params, core_number):
        # Read data from file
        with open(f"{path}\\{core_number}\\bc.in", "r") as f:
            lines = f.readlines()
            
        # set new values
        lines[46] = '3 0 4 0 1 		 % Final State: FSTYPE=read-in, OMEGA, NFSPRM, VLIMF, XCOORD\n'
        lines[47] = str(params.de) + "\n"
        lines[48] = str(params.re) + "\n"
        lines[49] = str(params.beta) + "\n"
        lines[50] = str(params.beta2) + "\n"

        # Write data to file
        with open(f"{core_number}\\bc.in", "w") as f:
            f.writelines(lines)

        # Run bc.bat
        subprocess.run(f"{path}\\{core_number}\\bc.bat", stdout=subprocess.DEVNULL, cwd=f'{path}\\{core_number}')
        subprocess.CompletedProcess(args=f"{path}\\{core_number}\\bc.bat", returncode=0)

        xs = []
        ys = []
        with open(f"{path}\\{core_number}\\bc.out", "r") as file:
            lines = file.readlines()
            iter = 0;
            for line in lines:
                if "Calculated Transition Intensity Coefficients" in line:
                    iter = iter + 3;
                    break
                else:
                    iter = iter + 1;
            for line in lines[iter:]:
                if "=====" in line:
                    break
                columns = line.split()
                x = float(columns[0])
                y = float(columns[2])
                xs.append(x)
                ys.append(y)
        return xs,ys
    
    def compare_spectra(self, params, ref_params):
        Solution.get_maxima(ref_params.de, ref_params.re, ref_params.beta, ref_params.beta2, ref_params.beta3, 0, 0, self.sigmaLevel)
        LU = LEVEL_utilizer()
        LU.readOutput(1)
        LU.prepareXAxis(0.05)
        spectr1 = LU.convolveAll(self.sigmaLevel)
        ref_bcont = self.generate_spectrum_line(ref_params, 0)
        Solution.get_maxima(params.de, params.re, params.beta, params.beta2, params.beta3, 0, 0, self.sigmaLevel)
        LU = LEVEL_utilizer()
        LU.readOutput(1)
        LU.prepareXAxis(0.05)
        spectr2= LU.convolveAll(self.sigmaLevel)
        bcont_spectrum = self.generate_spectrum_line(best.params, 0)
        plt.plot(ref_bcont[0], ref_bcont[1], color='darkgreen')
        plt.plot(LU.x, spectr1 * 2000000, color='darkblue')
        plt.plot(bcont_spectrum[0], bcont_spectrum[1], color='lawngreen')
        plt.plot(LU.x, spectr2 * 2000000, color='cyan')
        plt.xlim([42000, 49300])
        plt.show()
            
    def display_spectrum(self, other_ref, core_number):
        ref_data = self.generate_spectrum_line(self.reference_params_bcont, core_number)
        other_data = self.generate_spectrum_line(other_ref, core_number)
        plt.plot(ref_data[0], ref_data[1], color='r', label='ref')
        plt.plot(other_data[0], other_data[1], color='g', label='result')
        plt.xlim([44000, 50000])
        plt.show()
                
    def genetic_algorithm(self):
        self.bestResults="";        
        start_time = time.time()
        population = []
        print(f'Generating initial population...')
        # generate initial population
        nOfCores=8;
        
        popPerCore=self.population_size*2/nOfCores  
        popPerCore=int(popPerCore)
        populationCores=[];
        
        for i in range(nOfCores):
            populationCores.append([])
                
        threads = []
        for i in range(nOfCores):
            #print("new thread create", i)
            threads.append(Thread(target=self.generateInitialPop, args=(popPerCore,populationCores, i )))
            
        for t in threads:
            t.start()
        #print("threads len ", len(threads))
        for t in threads:
            t.join()
            
               
     
        for i in range(nOfCores):
            population.extend(populationCores[i])
        population.sort(key=lambda x: x.score)
        
        print(f'Initial population generated ({len(population)} elements), Time elapsed: {time.time() - start_time}s')
        print(f'Best score: {population[0]}')
        iter=0;    
        self.bestResults+=f'popul : {iter}  {population[0]} \n'
        iter=iter+1;
        
        # run genetic algorithm
        for i in range(self.generations):
            print(
                f'Generation {i + 1}, % of done: {round((i + 1) / self.generations * 100, 2)}%, '
                f'Time elapsed: {round(time.time() - start_time, 2)}s')
            population = self.selection(population)
            newpopulation=[];
            for i in range(0,5):
                newpopulation.append(population[i])
            self.mu =0, 
            self.sigma = self.population_size/10 
            
            popPerCore=self.population_size/nOfCores
            popPerCore=int(popPerCore)
            newPopulationCores=[];
            
            for i in range(nOfCores):
                newPopulationCores.append([])
                
            threads = []
            for i in range(nOfCores):
                print("new thread create", i)                
                threads.append(Thread(target=self.innerEvolution, args=(popPerCore,population,newPopulationCores, i )))            
            for t in threads:
                t.start()           
            for t in threads:
                t.join()
            
            for i in range(nOfCores):
                newpopulation.extend(newPopulationCores[i])
            newpopulation.sort(key=lambda x: x.score)
            
            #sorting of new population
            newpopulation = self.selection(newpopulation)           
            population=newpopulation;           
     
            print(f'Best score: {population[0]}')
            self.bestResults+=f'popul : {iter}  {population[0]} \n'
            iter=iter+1;
            self.display_spectrum(population[0].params, 1)
        print(f'Calculation done, Time elapsed: {round(time.time() - start_time, 2)}s')
        

        filename1 = str(int(time.time()))+".txt"
        writer = open(filename1, "a")
        writer.write(self.bestResults)
        writer.close()
        return population[0]
    


In [ ]:
#test on artificially generated spectrum (15 times to check stability)
for test in range(15):
    print(f'starting of test number {test}')
    refParams=  SolutionParameters(379.5, 3.605, 1.234, 0.08)  
    ref_spectrum_level = Solution.get_maxima(refParams.de, refParams.re, refParams.beta, refParams.beta2,0,0,1,10)
    ref_spectrum_bcont = Solution.get_maxima_bcont(refParams.de, refParams.re, refParams.beta,refParams.beta2,1, 30)

    s = Solution(ref_spectrum_level, ref_spectrum_bcont,refParams)
    print(s)
    s.population_size = 100
    s.generations = 10
    best = s.genetic_algorithm()
    print(f'Best score: {best}')
    end = time.time()
    print("Operation time {}".format(end - start))
    s.compare_spectra(best.params, refParams)
    print(f'Results of test number {test} \n')
    print(s.bestResults);
    print('\n\n\n\n\n\n\n')

starting of test number 0
Reference level: [  0.    62.   120.4  177.75 231.4  272.7 ]
Reference bcont: [0.0, 292.0, 584.0, 820.0, 1000.0, 1196.0, 1372.0, 1540.0, 1704.0, 1860.0, 2012.0, 2156.0, 2296.0, 2428.0, 2560.0, 2688.0, 2808.0, 2928.0, 3044.0, 3152.0, 3264.0, 3368.0, 3468.0, 3568.0, 3664.0, 3760.0, 3848.0, 3936.0, 4024.0, 4104.0, 4184.0]
temp: [  0.    75.4  146.2  216.95 281.  ]
Score: inf
Generating initial population...


In [1]:
#reading of maxima of experimental spectrum from files (without running of GA)
BC_Exp_Levels = open("BcontExpLevels.txt", "r")
dataBC = BC_Exp_Levels.read()
BC_Exp_Levels.close()
ref_spectrum_bcont=[];

dataBC=dataBC.split("\n")
for elem in dataBC:
    ref_spectrum_bcont.append(float(elem))    
    
    
Level_Exp_Levels = open("LevelExpLevels.txt", "r")
dataLEV = Level_Exp_Levels.read()
Level_Exp_Levels.close()
ref_spectrum_level=[];

dataLEV=dataLEV.split("\n")
for elem in dataLEV:
    ref_spectrum_level.append(float(elem))
    
print(ref_spectrum_bcont)
print(ref_spectrum_level)

[0.0, 667.0, 1099.0, 1478.0, 1798.0, 2084.0, 2353.0, 2592.0, 2821.0, 3042.0, 3240.0, 3423.0, 3607.0, 3792.0, 3952.0, 4101.0, 4248.0, 4380.0, 4517.0, 4645.0, 4765.0, 4891.0, 4993.0, 5098.0, 5197.0, 5297.0, 5392.0, 5487.0, 5573.0, 5654.0, 5731.0, 5804.0]
[0.0, 61.0, 123.0, 182.0, 234.0, 297.0]


In [ ]:
#reading of maxima of experimental spectrum from files and running of GA
refParams=  SolutionParameters(375.942, 3.458, 1.583,  0.0918) #defining of SolutionParameters (exact parameters values not important)

BC_Exp_Levels = open("BcontExpLevels.txt", "r")
dataBC = BC_Exp_Levels.read()
BC_Exp_Levels.close()
ref_spectrum_bcont=[];

dataBC=dataBC.split("\n")
for elem in dataBC:
    ref_spectrum_bcont.append(float(elem))   
    
Level_Exp_Levels = open("LevelExpLevels.txt", "r")
dataLEV = Level_Exp_Levels.read()
Level_Exp_Levels.close()
ref_spectrum_level=[];

dataLEV=dataLEV.split("\n")
for elem in dataLEV:
    ref_spectrum_level.append(float(elem))
    
print(ref_spectrum_bcont)
print(ref_spectrum_level)

s = Solution(ref_spectrum_level, ref_spectrum_bcont,refParams)
print(s)
s.population_size =150
s.generations = 10
best = s.genetic_algorithm()
print(f'Best score: {best}')
end = time.time()
print("czas całej operacji wyniósł {}".format(end - start))
s.compare_spectra(best.params, refParams)
print(f'wyniki testu \n')
print(s.bestResults);
print('\n\n\n\n\n\n\n')

[0.0, 667.0, 1099.0, 1478.0, 1798.0, 2084.0, 2353.0, 2592.0, 2821.0, 3042.0, 3240.0, 3423.0, 3607.0, 3792.0, 3952.0, 4101.0, 4248.0, 4380.0, 4517.0, 4645.0, 4765.0, 4891.0, 4993.0, 5098.0, 5197.0, 5297.0, 5392.0, 5487.0, 5573.0, 5654.0, 5731.0, 5804.0]
[0.0, 61.0, 123.0, 182.0, 234.0, 297.0]
Reference level: [0.0, 61.0, 123.0, 182.0, 234.0, 297.0]
Reference bcont: [0.0, 667.0, 1099.0, 1478.0, 1798.0, 2084.0, 2353.0, 2592.0, 2821.0, 3042.0, 3240.0, 3423.0, 3607.0, 3792.0, 3952.0, 4101.0, 4248.0, 4380.0, 4517.0, 4645.0, 4765.0, 4891.0, 4993.0, 5098.0, 5197.0, 5297.0, 5392.0, 5487.0, 5573.0, 5654.0, 5731.0, 5804.0]
temp: [  0.    59.55 116.2  167.6  218.8  257.5 ]
Score: 77.35
Generating initial population...


Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\Tomek\miniconda3\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "C:\Users\Tomek\miniconda3\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Tomek\AppData\Local\Temp\ipykernel_25336\3471167642.py", line 339, in generateInitialPop
  File "C:\Users\Tomek\AppData\Local\Temp\ipykernel_25336\3471167642.py", line 297, in generate_new_spectrum
  File "C:\Users\Tomek\AppData\Local\Temp\ipykernel_25336\3471167642.py", line 204, in get_maxima_bcont
IndexError: list assignment index out of range
